# Hello and welcome!

*What is this notebook about?*

This is my attempt to explore my personal diary and the posibilities of knwoledge discovery.

*Methodology*

I will try to iteratively (monthly) add new perspectives to this search for meaning.

*About my diary entries*

Unfortunately, I would feel rather uncomfortable to share my dairy entries -but i'll do share all my analysis results! (which is kind of ironic if you think that if I succeed in analysing my data, I will share all my "secrets" anyways 🥴)-, so here is some relevant information about them:
- I write very freely about a range of personal topics, thoughts and feelings.
- I write mostly in Spanish, some English, and some of my own language.
- I've gathered 21 months of entries so far.

## Results

...

In [5]:
# DATA INGESTION

import os
import pandas as pd
import re
pd.set_option('display.max_colwidth', -1)

#Read contents of journal archive into list for processing
folder='data/'
list_of_journals=[]
journal_files = os.listdir(folder)
print(journal_files)
for file in journal_files:
    full_file_path=folder+file
    list_of_journals.append(full_file_path)
    
#Create DataFrame shell to load data as we parse through journal
journal_df = pd.DataFrame(columns=['year', 'date', 'entry'])

#For each file(a single year), read in data.
for full_file_path in list_of_journals:
    journal_list_of_dicts = []
    journal_dict={}
    paragraph_list = []
    previous_date='NONE'
    journal_raw = open(full_file_path)
    
    #parse out specific year from text file for use in dataframe
    year = re.search(r'(\d{4})',full_file_path).group()
    
    #Read in journal line, by line and parse our dates to associate dates with specific entry  
    for paragraph in journal_raw.readlines():
        #Use pattern matching to parse out date, input can be various formats (May 17, 5/17, 5/17/2020, etc)
        pattern = r'(\d+(\/|-){1}\d+(\/|-)?\d{0,4})|((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s\d{1,2})'
        date_match = re.search(pattern, paragraph, re.IGNORECASE)
        if date_match and len(paragraph)<35: #length to prevent mid paragraph dates from being grabbed
            if len(journal_list_of_dicts)>0: #this is not the first instance of date, therefore we need to save date and list of paragraphs
                journal_dict = {'year':year,
                                'date': previous_date,
                                'entry': paragraph_list}
                journal_list_of_dicts.append(journal_dict)
                #save for use in next loop
                previous_date = date_match.group()
                #re_initialize list of paragraphs, and journal_dict
                paragraph_list = []
                journal_dict = {}
            else:
                if previous_date == 'NONE':
                #continue looping and save date, this is the first instance of saving a date - need to continue loop to graph paragraphs
                    previous_date = date_match.group()
                else:
                    #first instance and need to start list of dicts
                    journal_dict = {'date': previous_date,
                                    'year':year,
                                    'entry': paragraph_list}
                    journal_list_of_dicts.append(journal_dict)
                    previous_date = date_match.group(0)
                    #reset for next date
                    paragraph_list = []
                    journal_dict = {}
        else:
            #paragraph is not a date entry is continuing - must be saved to paragraph list
            paragraph_list.append(paragraph)
    #final entry
    journal_dict = {'date': previous_date,
                    'year':year,
                    'entry': paragraph_list}
    journal_list_of_dicts.append(journal_dict)
    #save to DF and concat to previous
    journal_df = pd.concat([pd.DataFrame(journal_list_of_dicts),journal_df], sort=False)

['diary-4.txt', 'diary-5.txt', 'diary-1.txt', 'diary-2.txt', 'diary-3.txt']


/var/folders/sd/c89_7ybj62s9w09y5w6qnvt80000gn/T/ipykernel_22575/459695175.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


AttributeError: 'NoneType' object has no attribute 'group'

## References:
- I heavily replicated this awesome notebook: https://github.com/datachico/data_mine_journal/blob/Updated_file/NLP_on_a_diary.ipynb